# More Standard library utilities

## Random Number generation

The C++ standard library has a fairly elaborate and sophisticated machinery for generation of pseudo-random numbers. These are in the header `<random>`.

In [ ]:
#pragma cling add_include_path("/p/project/training2312/local/include")
#include <iostream>
#include <random>
#include <map>

Conceptually, the random number generation has two important components:

    - A mathematical function to generate random uniformly distributed random bits
    - A mechanism to convert the randomly generated bits into random numbers distributed according to some distribution
    
The first part is the job of what are known as random "engines". These are mathematical sequences (therefore not really "random") where 

    - The correlation between subsequent values, second neighbour values, third, fourth ... neighbour values are (should be) all zero. (Independent)
    - The probability distribution of the generated values is the same along the generated sequence (Identically distributed)
    
The literature on pseudo-random number generation is quite rich with many high quality sequences. The C++ standard library random engines are based on *peer reviewed publications* on the subject at the time of the standardization of the random numbers in 2011. The available engines vary a lot in speed and randomness. The documentation of the available engines can be found at [the cppreference site](https://en.cppreference.com/w/cpp/numeric/random).

Unless you have special requirements based on your research area, you could use the default random engine of your compiler vendor, imaginatively named `std::default_random_engine`. I have used the Mersenne Twister engine, available as `std::mt19937_64`, scientific research work on off-lattice all-atom Monte Carlo simulations of proteins for over 17 years. So, let's try it!

How does one create a random number engine ?

In [ ]:
std::default_random_engine engine;

Now, just call this `engine` as if it were a function, with no arguments...

In [ ]:
engine()

In [ ]:
// again!
engine()

In [ ]:
// and again!
engine()

Every time you call `engine`, you get as the answer, the next number in its pseudo-random number sequence. The output is a 64 bit integer with 64 random bits. With this in hand, one could make a function so that instead of getting these long integers, we get uniformly distributed real numbers between 0 and 1. From there one can imagine generating different kinds of interesting distributions... normal distributions, Poisson distributions, Student's T distribution ... But that work has already been done for you!

Here is the code you need to write to get an adaptor that converts the uniform random bits we got from the engine into 64 bit integers (`long`) distributed according to the Poisson distribution, with a mean $\mu=9.5$.  

In [ ]:
const double mu = 9.5;
std::poisson_distribution<long> poisson{mu};

Our random variate with a Poisson distribution is now generated by calling `poisson` with the `engine` as its input!

In [ ]:
poisson(engine)

In [ ]:
// again
poisson(engine)

In [ ]:
// and again!
poisson(engine)

Is it any good ? Let's create a histogram by using `std::map`, generate 100000 values and compare with what's expected from the Poisson distribution...

In [ ]:
std::map<long, unsigned long> count; // our histogram

const auto N = 1000000UL;
for (auto i = 0UL; i < N; ++i) {
    auto r = poisson(engine); 
    // every time we execute the above, r will get a different value.
    // Those different values will be distributed according to the Poisson distribution. 
    ++count[r]; // Increment the count for r
}

We should compare the result with the Poisson distribution, $P(n) = \frac{\mu^n}{n!} \exp(-\mu)$. The first part, $\frac{\mu^n}{n!}$ can be calculated as $\frac{\mu}{n}\times\frac{\mu}{n-1}\times\dots\frac{\mu}{1}$. The exponential can then be multiplied later.

In [ ]:
// For comparison, we need the Poisson distribution function mapping (n, mu) -> prob
double poisfunc(long n, double mu) {
    double ans{1.0};
    while (n > 0) {
        ans *= (mu/n);
        --n;
    }
    ans *= std::exp(-mu);
    return ans;
}
// Although I strongly recommend that you use the new C++ function style, i.e.,
// auto poisfunc(long n, double mu) -> double
// the Jupyter-cling interpreted environment chokes and dies upon seeing that.
// Let's be kind to this tool here. But, when your tools can do real C++, you should
// prefer the new syntax.

In [ ]:
std::cout << "Outcome\tFrequency\tProbability\tExpected\n";
for (auto&& [outcome, frequency] : count) {
    std::cout << outcome << "\t" << frequency << "\t" 
        << frequency * 1.0 / N << "\t" << poisfunc(outcome, mu) << "\n";
}

For those of you who are not familiar with random experiments, and are worried about the "discrepancies" above: the experiment above is akin to the following. You know that a balanced coin toss should have 50-50 chance for heads and tails. If you do the experiment 10 times, do you *always* get 5 heads and 5 tails ? No. You might get 7-3 result once, 2-8 another time 5-5 another and so on. As the number of experiments becomes very very large, your results approach the theoretical values more and more. We performed one set of experiments above. Repeating it again will give us another set of frequencies. The larger the sample count, the more our observed frequenies resemble the theoretical expectations. Try generating $1000000$ samples instead of $1000$ and see if the numbers are closer!

You can now imagine what it would be like to use Mersenne twister engine `std::mt19937_64` or the Ranlux engine `std::ranlux_48` instead of your system default. You can also imagine what you would do if, instead of a Poisson distribution, you need a Normal distribution (`std::normal_distribution`), binomial distribution (`std::binomial_distribution`), Chi squared distribution (`std::chi_squared_distribution`), Gamma distribution, Cauchy distribution etc. No library other than what is built into the C++ standard library is required for any of this.

## Time measurements

How long did it take our generator above to generate 1 million random numbers ? We frequently need to do such measurements, and the standard library has the required tools in [the chrono library](https://en.cppreference.com/w/cpp/header/chrono). Let's illustrate by timing our above mentioned loop and then explain.

In [ ]:
#include <chrono>

In [ ]:
{
    std::map<long, unsigned long> count; // our histogram

    auto N = 1000000UL;
    auto t0 = std::chrono::steady_clock::now();
    for (auto i = 0UL; i < N; ++i) {
        auto r = poisson(engine); 
        // every time we execute the above, r will get a different value.
        // Those different values will be distributed according to the Poisson distribution. 
        ++count[r]; // Increment the count for r
    }
    auto t1 = std::chrono::steady_clock::now();
    std::cout << "Generation of distributed random numbers took "
        << std::chrono::duration<double>(t1 - t0).count() << " seconds\n";
}

There are different kinds of clocks in the `chrono` library. For most of your purposes, you can use the steady clock. The syntax is intuitive. The `now()` function gives us a time point. Subtracting them gives us a time duration, for which we chose `double` as the representation. The default unit for durations is seconds. So, we got our value in seconds. If we wanted to output milliseconds, we would do this:

In [ ]:
std::map<long, unsigned long> count; // our histogram

auto N = 1000000UL;
auto t0 = std::chrono::steady_clock::now();
for (auto i = 0UL; i < N; ++i) {
    auto r = poisson(engine); 
    // every time we execute the above, r will get a different value.
    // Those different values will be distributed according to the Poisson distribution. 
    ++count[r]; // Increment the count for r
}
auto t1 = std::chrono::steady_clock::now();
std::cout << "Generation of distributed random numbers took "
    << std::chrono::duration_cast<std::chrono::milliseconds>(t1 - t0).count() << " milliseconds\n";